In [118]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier,VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from imblearn.under_sampling import RandomUnderSampler

In [119]:
path = "PTSD.xlsx"
df = pd.read_excel(path)
df = df[~df["PCL_Strict3"].isna()]


In [120]:
features = [ "age", "highschool_diploma", "Hebrew", "dyslexia", "ADHD", "T1ETBE", "T1Acc1t",
                         "T1Acc1n", "T1bias", "T2Acc1t", "T2Acc1n", "T2bias", "state1", "state2", "trait1",
                         "trait2", "lot1", "lot2", "phq1", "phq2", "PCL1", "PCL2", "cd_risc1", "ptgi2",
                         "active_coping1", "planning1", "positive_reframing1", "acceptance1", "humor1",
                         "religion1", "emotional_support1", "instrumental_support1", "self_distraction1",
                         "denial1", "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1",
                         "active_coping2", "planning2", "positive_reframing2", "acceptance2", "humor2",
                         "religion2", "emotional_support2", "instrumental_support2","self_distraction2",
                         "denial2", "venting2", "substance_use2", "behavioral_disengagement2", "self_blame2",
                         "trauma_history8_1", "military_exposure_unit", "HML_5HTT", "HL_MAOA", "HML_NPY",
                         "COMT_Hap1_recode", "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5", "Ashken_scale",
                         "Sephar_scale", "Unknown", 'terror_p1', 'terror_i1', 'mva_p1', 'mva_i1',                   
                        'violent1', 'sexual1', 'rockets_p1', 'rockets_i1', 'trauma_history6_1',
                        'terror_p2','terror_i2','mva_p2', 'mva_i2', 'violent2', 'sexual2', 'rockets_p2', 'rockets_i2', 'trauma6t2', 'trauma8t2',
'military_exp18_1','military_exp18_t2', 'commanders18','commanders20','commanders22', 
't1bias_1_zero', 'state1_zero', 'trait1_zero', 'PHQ1_zero', 'PCL1_zero', 'depression_clinical2',
                        'avoid_bias', 'ptsd1_clini', 'avoidance_cop',
'clinical_depression15', 'avoidance_compa', 'resilience_compa', 'combat_compa', 'emotional_cop1n', 'avoidance_cop2', 'avoidance_cop']

In [121]:
numerical_features = [ "T1ETBE", "T1Acc1t", "T1Acc1n", "T1bias", "T2Acc1t","T2Acc1n", "T2bias", "state1", "state2",
                        "trait1", "trait2", "lot1", "lot2", "phq1", "phq2", "cd_risc1", "PCL1", "PCL2"]
categorical_features = [ "age", "highschool_diploma", "Hebrew", "dyslexia", "ADHD",   "ptgi2",
                    "active_coping1", "planning1", "positive_reframing1", "acceptance1", "humor1",
                    "religion1", "emotional_support1", "instrumental_support1", "self_distraction1",
                    "denial1", "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1",
                    "active_coping2", "planning2", "positive_reframing2", "acceptance2", "humor2",
                    "religion2", "emotional_support2", "instrumental_support2", "self_distraction2",
                    "denial2", "venting2", "substance_use2", "behavioral_disengagement2", "self_blame2",
                    "trauma_history8_1", "military_exposure_unit", "HML_5HTT", "HL_MAOA", "HML_NPY",
                    "COMT_Hap1_recode", "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5", "Ashken_scale",
                    "Sephar_scale", "Unknown", 'terror_p1', 'terror_i1', 'mva_p1', 'mva_i1',                   
                        'violent1', 'sexual1', 'rockets_p1', 'rockets_i1', 'trauma_history6_1',
                        'terror_p2','terror_i2','mva_p2', 'mva_i2', 'violent2', 'sexual2', 'rockets_p2', 'rockets_i2', 'trauma6t2', 'trauma8t2',
'military_exp18_1','military_exp18_t2', 'commanders18','commanders20','commanders22', 
't1bias_1_zero', 'state1_zero', 'trait1_zero', 'PHQ1_zero', 'PCL1_zero', 'depression_clinical2',
                        'avoid_bias', 'd_ptsd', 'ptsd1_clini', 'avoidance_cop',
'clinical_depression15', 'avoidance_compa', 'resilience_compa', 'combat_compa', 'emotional_cop1n', 'avoidance_cop2', 'avoidance_cop']
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
df[numerical_features] = imp.fit_transform(df[numerical_features])

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df[categorical_features] = imp.fit_transform(df[categorical_features])

X = df[features]
X = X - X.mean()

Y = df["PCL_Strict3"]

In [122]:
X["interaction_1"] = X["T1Acc1t"] * X["T2Acc1n"] * X["military_exposure_unit"]
X["interaction_2"] = X["T1Acc1n"] * X["T2Acc1t"] * X["military_exposure_unit"]

X["interaction_9"] = X["highschool_diploma"] * X["military_exp18_1"]
X["interaction_10"] = X["highschool_diploma"] * X["military_exp18_t2"]
X["interaction_11"] = X["highschool_diploma"] * X["military_exposure_unit"] * X['PCL1']
X["interaction_12"] = X["T1ETBE"] * X["military_exposure_unit"] * X['HML_5HTT']
X["interaction_13"] = X["T1ETBE"] * X['HML_5HTT']
X["interaction_14"] = X["T1ETBE"] * X['military_exposure_unit']

X["interaction_15"] = X["T1ETBE"] * X["PCL1"]

In [123]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1, random_state=271828, stratify=Y)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train, y_train, test_size = 0.05, random_state=271828, stratify=y_train)

In [132]:
pipe0 = Pipeline(steps=[
            ('SMOTE', RandomUnderSampler()),
                ('classifier', RandomForestClassifier(n_estimators=450))])
pipe1 = Pipeline(steps=[
            ('SMOTE', SMOTE()),
                ('classifier', RandomForestClassifier(n_estimators=450))])
pipe10 = Pipeline(steps=[
            ('SMOTE', RandomUnderSampler()),
                ('classifier', MLPClassifier(max_iter=1500, hidden_layer_sizes= (75, 50, 25, 5)))])
pipe01 = Pipeline(steps=[
            ('SMOTE', SMOTE()),
                ('classifier', MLPClassifier(max_iter=1500, hidden_layer_sizes = (75, 50, 25, 5)))])

In [133]:
pipe2 = Pipeline(steps=[
    ('PCA',  PCA(n_components = 0.9)),
                ('classifier', VotingClassifier(estimators=[('clf1', pipe0),('clf2', pipe1),('clf3', pipe10),('clf4', pipe01),
                                                           ('clf5', pipe0),('clf6', pipe1),('clf7', pipe10),('clf8', pipe01),
                                                           ('clf9', pipe0),('clf10', pipe1),('clf11', pipe10),('clf12', pipe01),
                                                            ('clf13', pipe0),('clf14', pipe1),('clf15', pipe10),('clf16', pipe01),
                                                            ('clf17', pipe0),('clf18', pipe1),('clf19', pipe10),('clf20', pipe01),
                                                            ('clf21', pipe0),('clf22', pipe1),('clf23', pipe10),('clf24', pipe01),
                                                            ('clf25', pipe0),('clf26', pipe1),('clf27', pipe10),('clf28', pipe01),
                                                            ('clf30', pipe0),('clf31', pipe1),('clf32', pipe10),('clf33', pipe01)]))])
params_grid = [{'PCA__n_components': [0.99, 0.95, 0.9]}]

In [134]:
cv = StratifiedKFold()

gs = GridSearchCV(pipe2, params_grid, cv=cv, scoring='f1')

C:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:624: FutureWarning: You should specify a value for 'n_splits' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(NSPLIT_WARNING, FutureWarning)


In [135]:
gs.fit(X_train_2, y_train_2)

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('PCA', PCA(copy=True, iterated_power='auto', n_components=0.9, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('classifier', VotingClassifier(estimators=[('clf1', Pipeline(memory=None,
     steps=[('SMOTE', RandomUnderSampler(random_state=None, ratio=None, replacement=False...warm_start=False))]))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'PCA__n_components': [0.99, 0.95, 0.9]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [136]:
gs.best_score_

0.228697149190837

In [137]:
gs.best_estimator_.steps[1]

('classifier', VotingClassifier(estimators=[('clf1', Pipeline(memory=None,
      steps=[('SMOTE', RandomUnderSampler(random_state=None, ratio=None, replacement=False,
           return_indices=False, sampling_strategy='auto')), ('classifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max...ue, solver='adam', tol=0.0001,
        validation_fraction=0.1, verbose=False, warm_start=False))]))],
          flatten_transform=None, n_jobs=None, voting='hard', weights=None))

In [138]:
f1_score(gs.predict(X_test_2), y_test_2)

0.28571428571428575